# Generate fake data with the same chunk size

In [ ]:
%%capture
!pip install git+https://github.com/jbusecke/pangeo-forge-recipes@dynamic_chunks_2

In [1]:
import fsspec
from pangeo_forge_recipes import aggregation, dynamic_target_chunks
import xarray as xr
import numpy as np
import os
import sys
sys.path.append('..')
import zarr_helpers

In [8]:
# Fake data directory
fake_data_dir = 'fake_data_with_chunks'

# Define dimensions
time_steps = 1
y = 600
x = 1200
multiple = 2 # how much do you want the dataset to grow by each iteration
n_multiples = 12

target_chunk_size = '5MB'

In [9]:
for n_multiple in range(n_multiples):
    if n_multiple == 0:
        size = y * x        
    else:
        size = y * x * multiple

    x = round(np.sqrt(2 * size))
    y = int(x/2)
    data = np.random.random(size=(time_steps, y, x))

    # Create Xarray datasets with dimensions and coordinates
    ds = xr.Dataset({
        'data': (['time', 'lat', 'lon'], data),
    }, coords={
        'time': np.arange(time_steps),
        'lat': np.linspace(-90, 90, y),
        'lon': np.linspace(-180, 180, x)
    })

    d = ds.to_dict(data=False)
    schema = aggregation.XarraySchema(
        attrs=d.get("attrs"),
        coords=d.get("coords"),
        data_vars=d.get("data_vars"),
        dims=d.get("dims"),
        chunks=d.get("chunks", {}),
    )
    try:
        target_chunks = dynamic_target_chunks.dynamic_target_chunks_from_schema(
            schema,
            target_chunk_size=target_chunk_size,
            target_chunks_aspect_ratio={'time': -1, 'lat': 1, 'lon': 2},
            size_tolerance=0.2
        )    
        print(target_chunks)
        ds = ds.chunk(target_chunks)    
        ds.to_zarr(f'{fake_data_dir}/store_lat_{y}x_lon_{x}.zarr', mode='w')
    except Exception as e:
        print(e)

target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
{'time': 1, 'lat': 600, 'lon': 1200}
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
{'time': 1, 'lat': 424, 'lon': 1697}
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
Could not find any chunk combinations satisfying the size constraint. Consider increasing size_tolerance or enabling allow_fallback_algo.
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
{'time': 1, 'lat': 848, 'lon': 848}
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
{'time': 1, 'lat': 1199, 'lon': 533}
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchunked_dims=['time']
Could not find any chunk combinations satisfying the size constraint. Consider increasing size_tolerance or enabling allow_fallback_algo.
target_chunks_aspect_ratio_chunked_only={'lat': 1, 'lon': 2}
unchun

In [10]:
# List all items in the directory
items = os.listdir(fake_data_dir)

# Loop through each item and open it with xarray if it's a Zarr store
for item in items:
    item_path = os.path.join(fake_data_dir, item)
    # Check if the item is a directory (Zarr stores are directories)
    if os.path.isdir(item_path):
        try:
            # Attempt to open the Zarr store using xarray
            ds = xr.open_zarr(item_path)
            print(f"Chunk size for {item}:")
            print(zarr_helpers.get_chunk_size(ds['data'])[2])
            print('-' * 80)  # Print a separator line
        except Exception as e:
            # Print an error message if unable to open the Zarr store
            print(f"Could not open {item} as a Zarr store. Error: {e}")


Chunk size for store_lat_4796x_lon_9592.zarr:
3.988372802734375
--------------------------------------------------------------------------------
Chunk size for store_lat_19182x_lon_38365.zarr:
4.039283752441406
--------------------------------------------------------------------------------
Chunk size for store_lat_600x_lon_1200.zarr:
5.4931640625
--------------------------------------------------------------------------------
Chunk size for store_lat_2398x_lon_4797.zarr:
4.875694274902344
--------------------------------------------------------------------------------
Chunk size for store_lat_1696x_lon_3392.zarr:
5.486328125
--------------------------------------------------------------------------------
Chunk size for store_lat_848x_lon_1697.zarr:
5.48956298828125
--------------------------------------------------------------------------------
Could not open .ipynb_checkpoints as a Zarr store. Error: No such file or directory: '/Users/aimeebarciauskas/github/developmentseed/tile-benc